In [36]:
import networkx as nx
import numpy as np
import pandas as pd
from scipy import sparse
from scipy.sparse import csr_matrix
from scipy.sparse import coo_matrix
from time import perf_counter

In [37]:
## filename = './datasets/facebook_combined.txt'
filename = './datasets/amazon0302.mtx'

In [38]:
t0 = perf_counter()
testgraph = nx.read_edgelist(filename, comments='#', delimiter=None, create_using=nx.Graph, nodetype=None, data=True, edgetype=None, encoding='utf-8')
t_load = perf_counter() - t0

In [39]:
## define the partition index
adj_matrix_dim = len(testgraph.nodes)
partition_num = 16 ## can be set a variable
partition_index = np.zeros(partition_num + 1, dtype=np.int32)
for i in range(partition_num):
    partition_index[i+1] = np.int32(adj_matrix_dim*(i+1)/partition_num)
print(partition_index)

[     0  16381  32763  49145  65527  81909  98291 114673 131055 147437
 163819 180201 196583 212965 229347 245729 262111]


In [40]:
## Get partition graph, maybe need to be parallel
G_part = []
for i in range(partition_num):
    G_part.append(nx.DiGraph())

for edge_list in testgraph.edges:
    for i in range(partition_num):
        if ((int(edge_list[1]) >= partition_index[i]) and (int(edge_list[1]) < partition_index[i+1])):
            G_part[i].add_edge(edge_list[0], edge_list[1])

for i in range(partition_num):
    print(G_part[i])

DiGraph with 24948 nodes and 63685 edges
DiGraph with 32980 nodes and 58881 edges
DiGraph with 37191 nodes and 58659 edges
DiGraph with 40421 nodes and 58511 edges
DiGraph with 43259 nodes and 58513 edges
DiGraph with 43063 nodes and 58400 edges
DiGraph with 45307 nodes and 58238 edges
DiGraph with 45502 nodes and 57807 edges
DiGraph with 45742 nodes and 57068 edges
DiGraph with 45664 nodes and 56118 edges
DiGraph with 45443 nodes and 54877 edges
DiGraph with 44896 nodes and 53621 edges
DiGraph with 43436 nodes and 52726 edges
DiGraph with 41324 nodes and 51254 edges
DiGraph with 37405 nodes and 50739 edges
DiGraph with 30108 nodes and 50690 edges


In [41]:
## triangle count in sub-graphs (need be parallel)
triangle_count = 0
node_triangle = np.zeros(len(testgraph.nodes))
for g_index in range(partition_num): ## could be executed parallelly in different machines
    for edge_list in testgraph.edges: ## traverse all the edges in the original graph
        if ((G_part[g_index].has_node(edge_list[0])) and (G_part[g_index].has_node(edge_list[1]))):
            srcSet = G_part[g_index].adj[edge_list[0]]
            dstSet = G_part[g_index].adj[edge_list[1]]
            for node_obj in srcSet:
                if node_obj in dstSet:
                    triangle_count += 1
                    node_triangle[int(edge_list[0])] += 1
                    node_triangle[int(edge_list[1])] += 1
                    node_triangle[int(node_obj)] += 1

print (triangle_count)
## print (node_triangle)


717711


In [42]:
testgraph_golden = nx.read_edgelist(filename, comments='#', delimiter=None, create_using=None, nodetype=None, data=True, edgetype=None, encoding='utf-8')
triangle_golden = nx.triangles(testgraph_golden)
result_golden = int(sum(triangle_golden.values())/3)
print (result_golden)
## print (triangle_golden)

717719
